# Ejercicio

Vamos a construir una BBDD en SQLite que modele, de forma muy estilizada, las necesidades de un administrador de cartera.

De forma muy sencilla un administrador tiene clientes, que tiene cuentas de inversión, a esas cuentas le hacen aportes y retiros de capital y con ese capital compran y venden activos financieros.

## Creación de la Base de Datos

In [1]:
import sqlite3

Si el archivo no existe, SQLite lo creará en la ruta especificada.

In [2]:
conn = sqlite3.connect('data/administrador.db') 

Se crea el objeto `cursor`.

In [3]:
cursor = conn.cursor()

Para habilitar las llaves foráneas.

In [4]:
cursor.execute("PRAGMA foreign_keys = 1")

La siguiente función nos permitirá ver las tablas de la base de datos.

In [5]:
import pandas as pd

In [6]:
def show_tables(conn):
    qry_tables = "SELECT name, sql FROM sqlite_master WHERE type = 'table'"
    return pd.read_sql(qry_tables, conn)

¿Qué hay ahora en la BBDD?

In [7]:
show_tables(conn)

,name,sql


Vemos que todavía no hay ninguna tabla.

## Tablas

La BBDD tendrá las siguientes tablas:

- clientes
- cuentas
- tipos de movimientos (compras, ventas, aportes, retiros)
- caja (cantidad de efectivo disponible en una cuenta)
- tipos de inversión (supondremos para simplificar que sólo se invierte en cuotas de fondos y acciones)
- inversiones (de una cuenta)
- precios de mercado (de las cuotas de fondos)

### Tabla `clientes`

Notar las restricciones de tipo `UNIQUE` en los campos `email` y `telefono`. Estas restricciones indican que estos campos deben ser únicos para cada registro, es decir, dos o más clientes **NO** pueden tener el mismo email o teléfono.

In [8]:
qry = "CREATE TABLE IF NOT EXISTS clientes ( \
id INTEGER PRIMARY KEY AUTOINCREMENT, \
nombre TEXT NOT NULL, \
apellido TEXT NOT NULL, \
email TEXT NOT NULL UNIQUE, \
telefono TEXT NOT NULL UNIQUE \
)"

In [9]:
cursor.execute(qry)

Verifiquemos el resultado.

In [10]:
show_tables(conn)

,name,sql
0,clientes,CREATE TABLE clientes ( id INTEGER PRIMARY KEY...
1,sqlite_sequence,"CREATE TABLE sqlite_sequence(name,seq)"


¿Y la tabla `sqlite_sequence`?

In [11]:
pd.read_sql("SELECT * FROM sqlite_sequence", conn)

,name,seq


Con esta función vamos a ir viendo si la tabla cambia más adelante.

In [12]:
def ss():
    return pd.read_sql("SELECT * FROM sqlite_sequence", conn)

### Tabla `cuentas`

La tabla `cuentas` tendrá una llave foránea hacia la tabla `clientes`. Este campo permite establecer de qué cliente es la cuenta.

In [13]:
qry = "CREATE TABLE IF NOT EXISTS cuentas ( \
id INTEGER PRIMARY KEY AUTOINCREMENT, \
nombre TEXT NOT NULL, \
id_cliente INTEGER, \
FOREIGN KEY (id_cliente) REFERENCES clientes (id) \
ON DELETE NO ACTION ON UPDATE NO ACTION \
)"

In [14]:
cursor.execute(qry)

In [15]:
show_tables(conn)

,name,sql
0,clientes,CREATE TABLE clientes ( id INTEGER PRIMARY KEY...
1,sqlite_sequence,"CREATE TABLE sqlite_sequence(name,seq)"
2,cuentas,CREATE TABLE cuentas ( id INTEGER PRIMARY KEY ...


In [16]:
ss()

,name,seq


### Tabla `tipo_movimiento`

Esta tabla registra los tipos de movimientos válidos de caja. Éstos serán `APORTE`, `RETIRO`, `COMPRA` y `VENTA`.

In [17]:
qry = "CREATE TABLE IF NOT EXISTS tipo_movimiento ( \
id INTEGER PRIMARY KEY AUTOINCREMENT, \
codigo TEXT NOT NULL UNIQUE \
)"

In [18]:
cursor.execute(qry)

In [19]:
show_tables(conn)

,name,sql
0,clientes,CREATE TABLE clientes ( id INTEGER PRIMARY KEY...
1,sqlite_sequence,"CREATE TABLE sqlite_sequence(name,seq)"
2,cuentas,CREATE TABLE cuentas ( id INTEGER PRIMARY KEY ...
3,tipo_movimiento,CREATE TABLE tipo_movimiento ( id INTEGER PRIM...


In [20]:
ss()

,name,seq


### Tabla `caja`

En esta tabla se registrarán los movimientos de caja de las cuentas.

In [21]:
qry = "CREATE TABLE IF NOT EXISTS caja ( \
id INTEGER PRIMARY KEY AUTOINCREMENT, \
id_cuenta INTEGER, \
codigo_movimiento TEXT NOT NULL, \
monto NUMERIC NOT NULL, \
FOREIGN KEY (id_cuenta) REFERENCES cuentas (id) \
ON DELETE NO ACTION ON UPDATE NO ACTION, \
FOREIGN KEY (codigo_movimiento) REFERENCES tipo_movimiento (codigo) \
ON DELETE NO ACTION ON UPDATE NO ACTION \
)"

In [22]:
cursor.execute(qry)

In [23]:
show_tables(conn)

,name,sql
0,clientes,CREATE TABLE clientes ( id INTEGER PRIMARY KEY...
1,sqlite_sequence,"CREATE TABLE sqlite_sequence(name,seq)"
2,cuentas,CREATE TABLE cuentas ( id INTEGER PRIMARY KEY ...
3,tipo_movimiento,CREATE TABLE tipo_movimiento ( id INTEGER PRIM...
4,caja,CREATE TABLE caja ( id INTEGER PRIMARY KEY AUT...


In [24]:
ss()

,name,seq


### Tabla `tipos_instrumentos`

En esta tabla se registran los tipos de instrumentos en los que se puede invertir. Para este ejemplo, sólo utilizaremos activos *P x Q*, es decir `ACCIONES`, `ETF`, `CUOTAS_FONDOS`.

In [25]:
qry = "CREATE TABLE IF NOT EXISTS tipos_instrumentos ( \
id INTEGER PRIMARY KEY AUTOINCREMENT, \
codigo TEXT UNIQUE \
)"

In [26]:
cursor.execute(qry)

In [27]:
show_tables(conn)

,name,sql
0,clientes,CREATE TABLE clientes ( id INTEGER PRIMARY KEY...
1,sqlite_sequence,"CREATE TABLE sqlite_sequence(name,seq)"
2,cuentas,CREATE TABLE cuentas ( id INTEGER PRIMARY KEY ...
3,tipo_movimiento,CREATE TABLE tipo_movimiento ( id INTEGER PRIM...
4,caja,CREATE TABLE caja ( id INTEGER PRIMARY KEY AUT...
5,tipos_instrumentos,CREATE TABLE tipos_instrumentos ( id INTEGER P...


In [28]:
ss()

,name,seq


### Tabla `instrumentos`

Esta tabla requiere una llave foránea hacia la tabla `tipos_instrumentos`.

In [29]:
qry = "CREATE TABLE IF NOT EXISTS instrumentos ( \
id INTEGER PRIMARY KEY AUTOINCREMENT, \
codigo TEXT UNIQUE, \
codigo_instrumento TEXT NOT NULL, \
FOREIGN KEY (codigo_instrumento) REFERENCES tipos_instrumentos (codigo) \
ON DELETE NO ACTION ON UPDATE NO ACTION \
)"

In [30]:
cursor.execute(qry)

In [31]:
show_tables(conn)

,name,sql
0,clientes,CREATE TABLE clientes ( id INTEGER PRIMARY KEY...
1,sqlite_sequence,"CREATE TABLE sqlite_sequence(name,seq)"
2,cuentas,CREATE TABLE cuentas ( id INTEGER PRIMARY KEY ...
3,tipo_movimiento,CREATE TABLE tipo_movimiento ( id INTEGER PRIM...
4,caja,CREATE TABLE caja ( id INTEGER PRIMARY KEY AUT...
5,tipos_instrumentos,CREATE TABLE tipos_instrumentos ( id INTEGER P...
6,instrumentos,CREATE TABLE instrumentos ( id INTEGER PRIMARY...


In [32]:
ss()

,name,seq


### Tabla `inversiones`

In [33]:
qry = "CREATE TABLE IF NOT EXISTS inversiones ( \
id INTEGER PRIMARY KEY AUTOINCREMENT, \
id_cuenta INTEGER NOT NULL, \
codigo_instrumento TEXT NOT NULL, \
monto NUMERIC NOT NULL, \
precio NUMERIC NOT NULL, \
FOREIGN KEY (id_cuenta) REFERENCES cuentas (id) \
ON DELETE NO ACTION ON UPDATE NO ACTION, \
FOREIGN KEY (codigo_instrumento) REFERENCES instrumentos (codigo) \
ON DELETE NO ACTION ON UPDATE NO ACTION \
)"

In [34]:
cursor.execute(qry)

In [35]:
show_tables(conn)

,name,sql
0,clientes,CREATE TABLE clientes ( id INTEGER PRIMARY KEY...
1,sqlite_sequence,"CREATE TABLE sqlite_sequence(name,seq)"
2,cuentas,CREATE TABLE cuentas ( id INTEGER PRIMARY KEY ...
3,tipo_movimiento,CREATE TABLE tipo_movimiento ( id INTEGER PRIM...
4,caja,CREATE TABLE caja ( id INTEGER PRIMARY KEY AUT...
5,tipos_instrumentos,CREATE TABLE tipos_instrumentos ( id INTEGER P...
6,instrumentos,CREATE TABLE instrumentos ( id INTEGER PRIMARY...
7,inversiones,CREATE TABLE inversiones ( id INTEGER PRIMARY ...


In [36]:
ss()

,name,seq


### Tabla `precios_mercado`

Esta tabla registrará los precios de mercado de los instrumentos. En este caso definiremos una restricción de tipo `UNIQUE` a una combinación de campos.

In [37]:
qry = "CREATE TABLE IF NOT EXISTS precios_mercado ( \
id INTEGER PRIMARY KEY AUTOINCREMENT, \
fecha TEXT NOT NULL, \
id_instrumento INTEGER NOT NULL, \
precio NUMERIC NOT NULL, \
FOREIGN KEY (id_instrumento) REFERENCES instrumentos (id) \
ON DELETE NO ACTION ON UPDATE NO ACTION, \
UNIQUE(fecha, id_instrumento) \
)"

In [38]:
cursor.execute(qry)

In [39]:
show_tables(conn)

,name,sql
0,clientes,CREATE TABLE clientes ( id INTEGER PRIMARY KEY...
1,sqlite_sequence,"CREATE TABLE sqlite_sequence(name,seq)"
2,cuentas,CREATE TABLE cuentas ( id INTEGER PRIMARY KEY ...
3,tipo_movimiento,CREATE TABLE tipo_movimiento ( id INTEGER PRIM...
4,caja,CREATE TABLE caja ( id INTEGER PRIMARY KEY AUT...
5,tipos_instrumentos,CREATE TABLE tipos_instrumentos ( id INTEGER P...
6,instrumentos,CREATE TABLE instrumentos ( id INTEGER PRIMARY...
7,inversiones,CREATE TABLE inversiones ( id INTEGER PRIMARY ...
8,precios_mercado,CREATE TABLE precios_mercado ( id INTEGER PRIM...


In [40]:
ss()

,name,seq


## Insertar Data

Vamos a insertar data en las distintas tablas que hemos definido.

### Tabla `clientes`

Recordemos los campos de la tabla.

In [41]:
pd.read_sql("SELECT * FROM clientes", conn)

,id,nombre,apellido,email,telefono


Para insertar data en una BBDD relacional se utiliza la instrucción `INSERT`. Notar que, dado que el campo `id` es `AUTOINCREMENT` no es necesario incluirlo.

In [42]:
qry = "INSERT INTO clientes (nombre, apellido, email, telefono) \
VALUES ('Alvaro', 'Diaz', 'alvaro@efaa.cl', 56955556666)"

In [43]:
cursor.execute(qry)

In [45]:
pd.read_sql("SELECT * FROM clientes", conn)

,id,nombre,apellido,email,telefono
0,1,Alvaro,Diaz,alvaro@efaa.cl,56955556666


**HABLAR DEL COMMIT**

In [47]:
conn.commit()

Podemos también ingresar más de un registro a la vez.

In [48]:
qry = "INSERT INTO clientes (nombre, apellido, email, telefono) \
VALUES ('Ricardo', 'Gomez', 'ricardo.gomez@efaa.cl', 56955551234), \
('Thomas', 'Macmillan', 'thomas.macmillan@efaa.cl', 56955554321)"

In [49]:
cursor.execute(qry)
conn.commit()

In [50]:
pd.read_sql("SELECT * FROM clientes", conn)

,id,nombre,apellido,email,telefono
0,1,Alvaro,Diaz,alvaro@efaa.cl,56955556666
1,2,Ricardo,Gomez,ricardo.gomez@efaa.cl,56955551234
2,3,Thomas,Macmillan,thomas.macmillan@efaa.cl,56955554321


Recordemos que el campo `email` tiene una restricción `UNIQUE`, veamos qué ocurre si, por error, ingresamos un email repetido.

In [51]:
qry = "INSERT INTO clientes (nombre, apellido, email, telefono) \
VALUES ('Oliver', 'Mohr', 'alvaro@efaa.cl', 56955553141)"

In [52]:
cursor.execute(qry)

IntegrityError: UNIQUE constraint failed: clientes.email

In [53]:
pd.read_sql("SELECT * FROM clientes", conn)

,id,nombre,apellido,email,telefono
0,1,Alvaro,Diaz,alvaro@efaa.cl,56955556666
1,2,Ricardo,Gomez,ricardo.gomez@efaa.cl,56955551234
2,3,Thomas,Macmillan,thomas.macmillan@efaa.cl,56955554321


In [54]:
ss()

,name,seq
0,clientes,3


### Tabla `cuentas`

Recordemos los campos de esta tabla.

In [55]:
pd.read_sql("SELECT * FROM cuentas", conn)

,id,nombre,id_cliente


Vamos a insertar algunas cuentas.

In [56]:
cuentas = [
    ('CUENTA1', 1),
    ('CUENTA2', 1),
    ('CUENTA1', 2),
    ('CUENTA1', 3)
]

Con este comando podemos insertar todas las cuentas con una sola instrucción.

In [62]:
cursor.executemany("INSERT INTO cuentas (nombre, id_cliente) VALUES (?, ?)", cuentas)
conn.commit()

In [63]:
pd.read_sql("SELECT * FROM cuentas", conn)

,id,nombre,id_cliente
0,9,CUENTA1,1
1,10,CUENTA2,1
2,11,CUENTA1,2
3,12,CUENTA1,3


¿Qué pasa si tratamos de insertar una cuenta para un cliente que no existe?

In [64]:
cursor.execute("INSERT INTO cuentas (nombre, id_cliente) VALUES ('CUENTA1', 4)")

IntegrityError: FOREIGN KEY constraint failed

In [66]:
ss()

,name,seq
0,clientes,3
1,cuentas,12


### Tabla `tipo_movimiento`

Esta es una tabla con data estática. Vamos a insertar los 4 tipos de movimientos que indicamos al inicio.

In [73]:
pd.read_sql("SELECT * FROM tipo_movimiento", conn)

,id,codigo
0,1,APORTE
1,2,RETIRO
2,3,COMPRA
3,4,VENTA


In [67]:
movimientos = [
    ('APORTE',), ('RETIRO',), ('COMPRA',), ('VENTA',)
]

In [69]:
cursor.executemany("INSERT INTO tipo_movimiento (codigo) VALUES (?)", movimientos)
conn.commit()

IntegrityError: UNIQUE constraint failed: tipo_movimiento.codigo

### Tabla `caja`

Vamos a hacerle un aporte de 10 MM CLP a la `CUENTA1` de Ricardo.

In [76]:
pd.read_sql("SELECT * FROM caja", conn).style.format({'monto': '{:,.0f}'})

,id,id_cuenta,codigo_movimiento,monto
0,1,12,APORTE,"10,000,000"


In [74]:
qry = "INSERT INTO caja (id_cuenta, codigo_movimiento, monto) VALUES (12, 'APORTE', 10000000)"

In [75]:
cursor.execute(qry)
conn.commit()

### Tabla `tipos_instrumentos`

Esta es una tabla con data estática. Vamos a insertar dos tipos de instrumentos.

In [80]:
pd.read_sql("SELECT * FROM tipos_instrumentos", conn)

,id,codigo
0,1,ACCIONES
1,2,CUOTA_FONDOS
2,3,ETF


In [78]:
tipos_instrumentos = [
    ('ACCIONES',), ('CUOTA_FONDOS',), ('ETF',)
]

In [79]:
cursor.executemany("INSERT INTO tipos_instrumentos (codigo) VALUES (?)", tipos_instrumentos)
conn.commit()

### Tabla `instrumentos`

In [84]:
pd.read_sql("SELECT * FROM instrumentos", conn)

,id,codigo,codigo_instrumento
0,1,COPEC,ACCIONES
1,2,MMKT_LV,CUOTA_FONDOS
2,3,SP500,ETF


In [82]:
instrumentos = [
    ('COPEC', 'ACCIONES'),
    ('MMKT_LV', 'CUOTA_FONDOS'),
    ('SP500', 'ETF'),
]

In [83]:
cursor.executemany("INSERT INTO instrumentos (codigo, codigo_instrumento) VALUES (?, ?)", instrumentos)
conn.commit()

### Compra de un Instrumento

Aquí vamos a ver un ejemplo de la utilidad de usar `commit`. Al ingresar una compra de un instrumento, se deben realizar dos inserciones de datos:

- Un nuevo registro en la tabla `inversiones`
- Un nuevo movimiento en la tabla `caja` que refleje que se ha utilizado el saldo.

Ambas operaciones están envueltas de forma explícita en una `TRANSACTION`, de modo que si cualquier parte de ésta falla, no quedará incompleta y todo se puede reversar.

In [85]:
id_cuenta = 12
instrumento = 'COPEC'
codigo = 'COMPRA'
numero = 1000
precio = 7150

In [86]:
qry_caja = "INSERT INTO caja (id_cuenta, codigo_movimiento, monto) VALUES (?, ?, ?)"

In [87]:
qry_inversion = "INSERT INTO inversiones (id_cuenta, codigo_instrumento, monto, precio) \
VALUES (?, ?, ?, ?)"

In [88]:
try:
    cursor.execute("BEGIN TRANSACTION")
    cursor.execute(qry_caja, [id_cuenta, codigo, numero * precio if codigo == 'VENTA' else -numero * precio])
    cursor.execute(qry_inversion, [id_cuenta, instrumento, numero, precio])
    conn.commit()
except Exception as e:
    print(str(e))
    cursor.execute("ROLLBACK")

In [89]:
pd.read_sql("SELECT * FROM caja", conn)

,id,id_cuenta,codigo_movimiento,monto
0,1,12,APORTE,10000000
1,2,12,COMPRA,-7150000


In [90]:
pd.read_sql("SELECT * FROM inversiones", conn)

,id,id_cuenta,codigo_instrumento,monto,precio
0,1,12,COPEC,1000,7150


## Cerrar Conexión

In [91]:
conn.close()